#### Imports

In [66]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons, make_circles

In [67]:
from mpl_toolkits.mplot3d import Axes3D

%matplotlib inline
%config InlineBackend.figure_format='retina'

In [68]:
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader, TensorDataset

#### Create datasets

In [69]:
nTot = 1000
circles_data, circles_labels = make_circles(n_samples = nTot)

In [70]:
# idx_0 = np.where(circles_labels==0)
# idx_1 = np.where(circles_labels==1)
# plt.scatter(circles_data[idx_0,0],circles_data[idx_0,1])
# plt.scatter(circles_data[idx_1,0],circles_data[idx_1,1])
# plt.axis('equal')
# plt.show()

#### Split datat into train and test

In [71]:
fracTrain = 0.8
fracTest = 1 - fracTrain

nTrain = int(nTot*fracTrain)
nTest = nTot - nTrain

In [72]:
nRounds = 3

In [73]:
repsTrain_solo = circles_data[:nTrain]
repsTest_solo = circles_data[nTrain:]

labelsTrain = circles_labels[:nTrain].astype(float)
labelsTest = circles_labels[nTrain:].astype(float)

In [74]:
dimProj = 10
A = np.random.rand(2,dimProj)

repsTrain_proj = repsTrain_solo@A
repsTest_proj = repsTest_solo@A

#### Make sequences

In [75]:
def makeSequences(dataset,Trounds=1):
    if Trounds==1:
        return np.expand_dims(dataset,1)
    elif Trounds>1:
        numSamps, numFeats = dataset.shape
        dataset_sequential = np.zeros((numSamps,Trounds,numFeats))
        for kk in range(Trounds):
            dataset_sequential[:,kk,:] = dataset
        return dataset_sequential

In [76]:
repsTrain = makeSequences(repsTrain_proj,nRounds)
repsTest = makeSequences(repsTest_proj,nRounds)

#### Convert data to tensors

In [77]:
dataTrain = Variable(torch.from_numpy(repsTrain)).requires_grad_(True)
yTrain = Variable(torch.from_numpy(labelsTrain)).requires_grad_(True)

dataTest = Variable(torch.from_numpy(repsTest)).requires_grad_(True)
yTest = Variable(torch.from_numpy(labelsTest)).requires_grad_(True)

#### Train and test loaders

In [78]:
## create dataset and dataloader
tensorTrainData = TensorDataset(dataTrain,yTrain)
tensorTestData = TensorDataset(dataTest,yTest)

bs = 128 ## batch size
train_loader = DataLoader(tensorTrainData, batch_size=bs, shuffle=True)
test_loader = DataLoader(tensorTestData, batch_size=bs, shuffle=False)

#### Test for CUDA

In [79]:
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('No GPU, training on CPU')
else:
    print('GPU found, training on GPU')

GPU found, training on GPU


#### RNN

In [80]:
class RNNpredictor(nn.Module):
    def __init__(self, seq_len=3, n_features=10, hidden_dim=5):
        super(RNNpredictor, self).__init__()
        
        ##Encoder
        self.layer1 = nn.RNN(input_size = n_features, hidden_size = hidden_dim,
                                 num_layers = 1, batch_first = True, bidirectional = False)
        
        ## FC layer
        self.fc = nn.Linear(hidden_dim, 2)
        
    def forward(self, x):
        
        ## layer 1
        x, lastHidden = self.layer1(x)
        
        classifyOp = self.fc(lastHidden)
        
        return classifyOp

#create the NN
model = RNNpredictor()
print(model)

#move tensors to GPU if available
if train_on_gpu:
    model.cuda()

RNNpredictor(
  (layer1): RNN(10, 5, batch_first=True)
  (fc): Linear(in_features=5, out_features=2, bias=True)
)


#### Count number of parameters

In [81]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
count_parameters(model)

97

#### Trial pass

In [83]:
for data, target in train_loader:
    if train_on_gpu:
        data, target = data.float().cuda(), target.float().cuda()
        op = model(data)

print(data.shape)
print(op.shape)

torch.Size([32, 3, 10])
torch.Size([1, 32, 2])


#### Hebbian update - MLP mapping

#### MLP Architecture

In [ ]:
class MLPAE(nn.Module):
    def __init__(self):
        super(MLPAE, self).__init__()
        
        ##Encoder
        self.layer1 = nn.Linear(10, 5)
        self.layer2 = nn.Linear(5,2)
        self.layer3 = nn.Linear(2, 5)
        self.layer4 = nn.Linear(5,10)
        
        self.dropout = nn.Dropout(0.3)
        
    def forward(self, x):
        
        ## layer 1
        x = F.relu(self.layer1(x))
        x = self.dropout(x)
        x = F.relu(self.layer2(x))
        x_latent = self.dropout(x)
        x = F.relu(self.layer3(x))
        x = self.dropout(x)
        x_recon = self.layer4(x)
        
        return x_recon, x_latent

#create the NN
model = MLPAE()
print(model)

#move tensors to GPU if available
if train_on_gpu:
    model.cuda()